Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


c:\Users\Farrukh Mumtaz\anaconda3\envs\tf_env\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3349: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


Epoch 1/10
  5/718 [..............................] - ETA: 45:13 - loss: 22.1904 - age_output_loss: 20.2362 - facial_expression_output_loss: 1.9542 - age_output_accuracy: 0.5714 - facial_expression_output_accuracy: 0.2062

KeyboardInterrupt: 

In [7]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2

# Load the CK+ dataset
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of the data will be used for validation
)

# Use flow_from_directory to load images, assuming the images are organized by class in subdirectories
train_generator = datagen.flow_from_directory(
    'C:/Users/Farrukh Mumtaz/Downloads/ck+',  # Update the path as needed
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Set as training data
)

validation_generator = datagen.flow_from_directory(
    'C:/Users/Farrukh Mumtaz/Downloads/ck+',  # Update the path as needed
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Set as validation data
)

# Create the model based on MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for facial expression prediction only
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
facial_expression_output = Dense(7, activation='softmax', name='facial_expression_output')(x)  # Adjust the number of classes as needed

model = Model(inputs=base_model.input, outputs=[facial_expression_output])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with 5 epochs
history = model.fit(train_generator, validation_data=validation_generator, epochs=10)

# Save the trained model
model.save('facial_expression_model1.h5')

# Evaluate the model on the validation set
validation_loss, validation_accuracy = model.evaluate(validation_generator)

print('Validation Loss:', validation_loss)
print('Validation Accuracy:', validation_accuracy)

# Predict facial expression on a new image
new_image_path = 'img3.jpg'  # Update to your image path
new_image = cv2.imread(new_image_path)
new_image = cv2.resize(new_image, (224, 224))
new_image = preprocess_input(new_image)

predicted_facial_expression = model.predict(np.expand_dims(new_image, axis=0))
predicted_facial_expression = np.argmax(predicted_facial_expression[0])

print('Predicted Facial Expression:', predicted_facial_expression)

Found 788 images belonging to 7 classes.
Found 193 images belonging to 7 classes.
Epoch 1/10
25/25 [==============================] - 98s 4s/step - loss: 1.1352 - accuracy: 0.6421 - val_loss: 5.3001 - val_accuracy: 0.3368
Epoch 2/10
25/25 [==============================] - 102s 4s/step - loss: 0.4201 - accuracy: 0.8490 - val_loss: 10.1776 - val_accuracy: 0.3212
Epoch 3/10
25/25 [==============================] - 104s 4s/step - loss: 0.2522 - accuracy: 0.9239 - val_loss: 12.5084 - val_accuracy: 0.3109
Epoch 4/10
25/25 [==============================] - 104s 4s/step - loss: 0.1498 - accuracy: 0.9467 - val_loss: 13.0363 - val_accuracy: 0.3575
Epoch 5/10
25/25 [==============================] - 103s 4s/step - loss: 0.1505 - accuracy: 0.9645 - val_loss: 14.6576 - val_accuracy: 0.2953
Epoch 6/10
25/25 [==============================] - 102s 4s/step - loss: 0.1816 - accuracy: 0.9480 - val_loss: 15.2708 - val_accuracy: 0.2435
Epoch 7/10
25/25 [==============================] - 102s 4s/step - l

In [5]:
import tensorflow as tf
import cv2
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('facial_expression_model.h5')

# Define the classes (adjust based on your dataset)
classes = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise', 'Neutral']  # Update this list as needed

# Start video capture from the camera
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the image
    resized_frame = cv2.resize(frame, (224, 224))
    normalized_frame = preprocess_input(resized_frame)
    input_frame = np.expand_dims(normalized_frame, axis=0)

    # Make predictions
    predictions = model.predict(input_frame)
    predicted_class = np.argmax(predictions[0])
    predicted_label = classes[predicted_class]

    # Display the resulting frame with the predicted label
    cv2.putText(frame, f'Predicted: {predicted_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.imshow('Live Camera Feed', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()


In [16]:
import tensorflow as tf
import cv2
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('facial_expression_model1.h5')

# Define the classes (adjust based on your dataset)
classes = ['anger', 'contempt', 'disgust', 'fear', 'happy','sadness', 'surprise']  # Update this list as needed

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start video capture from the camera
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    # Process each detected face
    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Extract the face region for prediction
        face_roi = frame[y:y + h, x:x + w]
        resized_face = cv2.resize(face_roi, (224, 224))
        normalized_face = preprocess_input(resized_face)
        input_face = np.expand_dims(normalized_face, axis=0)

        # Make predictions
        predictions = model.predict(input_face)
        predicted_class = np.argmax(predictions[0])
        predicted_label = classes[predicted_class]

        # Display the predicted label
        cv2.putText(frame, predicted_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Display the resulting frame
    cv2.imshow('Live Camera Feed', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()


In [15]:
import tensorflow as tf
import cv2
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('facial_expression_model1.h5')

# Define the classes (adjust based on your dataset)
classes = ['anger', 'contempt', 'disgust', 'fear', 'happy','sadness', 'surprise']  # Update this list as needed

# Load Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start video capture from the camera
cap = cv2.VideoCapture(0)  # Use 0 for the default camera

# Prediction frame skip counter
frame_counter = 0
frame_skip = 5  # Predict every 5 frames

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

    # Process each detected face
    for (x, y, w, h) in faces:
        # Draw rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Extract the face region for prediction
        face_roi = frame[y:y + h, x:x + w]
        resized_face = cv2.resize(face_roi, (224, 224))
        normalized_face = preprocess_input(resized_face)
        input_face = np.expand_dims(normalized_face, axis=0)

        # Predict every frame_skip frames
        if frame_counter % frame_skip == 0:
            predictions = model.predict(input_face)
            predicted_class = np.argmax(predictions[0])
            predicted_label = classes[predicted_class]
            confidence = np.max(predictions[0])

            # Only display label if confidence is above a threshold (e.g., 0.6)
            if confidence > 0.6:
                cv2.putText(frame, f'{predicted_label} ({confidence:.2f})', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Display the resulting frame
    cv2.imshow('Live Camera Feed', frame)

    # Increment frame counter
    frame_counter += 1

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()
